# Тестовое задание для стажеров DS по разработке PD модели

<b>Задание:</b> Необходиом разработать модель предсказания дефолта. Даны финансовые данные 32 395 компаний (выручка, активы, пассивы и т.д). Результат будет оцениваться на скрытой выборке из 200 компаний по метрике качества accuracy_score

<b>Цель задания</b> научиться / показать знания в разработке моделей оценки риска - probability of default. Задание приближено к реальному, данные синтетические. Лидерборд для оценки собственного решения и геймификации процесса обучения. Первое место в лидерборде не явлется целью. Цель - правильно разарботанная модель, удовлетворяющая требованиям банковской системы. В случае возникновения вопросов, сотрудники блока Риски готовы проконсультировать и дать обратную связь на модель (телеграм чат).

<b>Метрика качества</b> accuracy_score 

### Импортируем библиотеки

In [ ]:
!pip install pandas-profiling --upgrade

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
!pip install skrebate

In [ ]:
from skrebate import ReliefF

### Подгружаем данные

In [ ]:
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data-train.csv
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data-test.csv
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data-desc.csv

--2021-02-09 21:12:01--  https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5329268 (5.1M) [text/plain]
Saving to: ‘PD-data-train.csv’

PD-data-train.csv   100%[===================>]   5.08M  20.4MB/s    in 0.2s    

2021-02-09 21:12:03 (20.4 MB/s) - ‘PD-data-train.csv’ saved [5329268/5329268]

--2021-02-09 21:12:03--  https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28259 (28K) [text/plain]
Saving to: ‘

In [ ]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

### Описание данных

In [ ]:
desc_df

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)
5,ar_sale_profit,Profit from sales (at the end of the last year),Прибыль от реализации (на конец последнего года)
6,ar_balance_of_rvns_and_expns,Balance of income and expenses (at the end of ...,Баланс доходов и расходов (на конец последнего...
7,ar_profit_before_tax,Gross Profit (end of last year),Валовая прибыль (на конец последнего года)
8,ar_taxes,Current income tax (at the end of the last year),Текущий на налог на прибыль (на конец последне...
9,ar_other_profit_and_losses,Other income and expenses (at the end of the l...,Прочие доходы и расходы (на конец последнего г...


Семплы и размеры выборок

In [ ]:
train_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256,135,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,33,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,27,27,[1-100],10000.0,2,0,0,0,0


In [ ]:
train_df.shape

(32395, 37)

In [ ]:
train_df['default_12m'].value_counts()

0    30304
1     2091
Name: default_12m, dtype: int64

Наблюдается довольно сильный дисбаланс классов

In [ ]:
test_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,196,39103000.0,38913000.0,38913000.0,0.0,0.0,190000.0,141000.0,331000.0,284000.0,0.0,47000.0,0.0,6009000.0,2451000.0,3558000.0,0.0,677000.0,47000.0,1178000.0,6130000.0,0.0,3000000.0,3130000.0,0.0,156,135,3,3,3,[1-100],100000.0,1,0,0,0
1,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216,138,5,5,5,[1-100],100000.0,1,0,0,0
2,2813,54174000.0,50929000.0,50929000.0,0.0,0.0,3245000.0,-1278000.0,1967000.0,500000.0,0.0,1467000.0,0.0,8175000.0,8037000.0,138000.0,0.0,237000.0,1467000.0,6835000.0,3906000.0,0.0,0.0,3906000.0,0.0,244,138,20,20,20,[1-100],250000.0,2,0,0,0
3,4385,1904000.0,1679000.0,1679000.0,0.0,0.0,225000.0,-88000.0,137000.0,0.0,0.0,137000.0,0.0,5790000.0,4314000.0,1476000.0,0.0,35000.0,137000.0,5691000.0,540000.0,0.0,500000.0,40000.0,0.0,272,139,9,9,9,[1-100],10000.0,2,0,0,0
4,6479,225584000.0,210685000.0,210685000.0,0.0,0.0,14899000.0,-12715000.0,2184000.0,445000.0,0.0,1739000.0,0.0,65485000.0,46006000.0,19479000.0,0.0,956000.0,1739000.0,12408000.0,47017000.0,0.0,16215000.0,30802000.0,0.0,254,139,3,3,3,(100-500],1700598.0,84,0,0,0


In [ ]:
test_df.shape

(200, 36)

Количество разных типов фичей

In [ ]:
train_df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [ ]:
train_df['default_12m'].values.mean()

0.06454699799351751

In [ ]:
train_df.record_id.nunique()

32395

32395 уникальных наблюдений

### Немного поисследуем данные

Создадим автоматические отчеты через pandas profiling для тестового и тренировочного датасетов. Из-за того, что колаб вылетает после формирования отчета, скачаем их в виде html файлов

In [ ]:
report = pandas_profiling.ProfileReport(train_df)
report.to_file(output_file='train_report.html')

Summarize dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
report_test = pandas_profiling.ProfileReport(test_df)
report_test.to_file(output_file='test_report.html')

Summarize dataset:   0%|          | 0/49 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Мои модели
---

Можно наблюдать, что по фичам с 0 по 23 (без учета record_id) 46.8% пропусков в трейне и 59% пропусков в тесте. При этом пропуски в одних и тех же строчках сразу по всем показателям. Очевидно, пропуски связаны с тем, что для данных компаний просто отсутствуют данные финансовой отчетности. Однако для прогнозирования дефолта данные показатели могут оказаться важны. 

Было решено попробовать разделить данные на 2 части (с пропусками и без) и для каждой из них строить отдельную модель.


---
Избавимся от фич ['ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg'], т.к. они крайне малоинформативны (можно убедиться в отчете, в трейне они практически константны, в тесте - полностью константы). Также избавимся сразу от 'record_id'. Кроме этого избавимся от ['head_actual_age', 'cap_actual_age'], т.к. они являются полными дубликатами 'adr_actual_age'

In [ ]:
to_drop = ['record_id', 'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg', 'head_actual_age', 'cap_actual_age']
train_df.drop(to_drop, axis=1, inplace=True)

Также закодируем категориальную переменную 'ul_staff_range', для начала при помощи простого label-кодирования, но потом можно попробовать что-то поинтереснее (если в этом будет особый смысл)

In [ ]:
train_df['ul_staff_range'].value_counts()

[1-100]      29281
(100-500]     2757
> 500          357
Name: ul_staff_range, dtype: int64

In [ ]:
# Закодируем вручную, чтобы случайно не потерять смысл возрастания количества сотрудников

def simple_staff_coder(x):
    code_dict = {'[1-100]': 0, '(100-500]': 1, '> 500': 2}
    return code_dict[x]

train_df['ul_staff_range'] = train_df['ul_staff_range'].apply(simple_staff_coder)

train_df['ul_staff_range'].value_counts()

0    29281
1     2757
2      357
Name: ul_staff_range, dtype: int64

Теперь разделим данные на 2 части:

In [ ]:
# 0 - cо всеми фичами, 1 - без данных фин. отчетности

X_0 = train_df.dropna()
X_1 = train_df.loc[train_df.iloc[:, :24].isna().any(axis=1), train_df.columns[24:]]

y_0 = X_0['default_12m']
y_1 = X_1['default_12m']

X_0 = X_0.drop('default_12m', axis=1)
X_1 = X_1.drop('default_12m', axis=1)

In [ ]:
# Проверим, не потерялись ли какие-нибудь данные

X_0.shape[0] + X_1.shape[0] == train_df.shape[0]

True

На этом обработка трейна пока закончена. Будем обучаться сразу на всей выборке с кросс-валидацией (без дополнительного train_test_split).

## Генерация дополнительнич фич для 0 группы

13 дополнительных факторов: **nf_1, nf_2, ..., nf_13**

**Факторы из модели дискриминатного анализа Альтмана: (1-5)**
- доля чистого оборотного капитала в сумме активов
- нераспределенная прибыль / активы (рентабельность активов)
- отношение прибыли до уплаты процентов и налогов к величине активов
- отношение собственного капитала к объему заемных средств
- выручка / активы (оборачиваемость активов)

**Факторы из модели Чессера (адаптир. под РФ, без повторяющ., те, что можно рассчитать): (6-9)**
- денежные средства + краткосрочные фин. вложения)/оборотные + внеоборотные активы
- выручка от продаж/денежные средства + краткосрочные фин. вложения)
- долгосрочные обязательства + краткосрочные обязательства)/оборотные + внеоборотные активы)
- оборотный капитал/выручка от продаж

**Показатели для установления неудовлетворительной или удовлетворительной структуры баланса: (10-13)**
- Коэффициент абсолютной ликвидности: денежныесредства + краскосрочные фин. вложения / краткосрочные обязательства
- Коэффициент срочной ликвидности: ден.средства + краскоср.фин.вложения + деб. задолж. / краткосрочные обязательства
- Коэффициент текущей ликвидности: оборотные активы / краткосрочные обязательства
- Коэффициент обеспеченности собственными средствами: собственные оборотные средства / оборотные активы

In [ ]:
# Напишем функцию, чтобы затем легко добавить новые факторы в тестовый датасет

def get_new_features(df_original):
    df = df_original.copy()
    # краткоср. обязательства:
    short_term_lialabilities = df.ab_short_term_borrowing	+ df.ab_accounts_payable + df.ab_other_borrowings 
    # активы:
    assets = df.ab_mobile_current_assets + df.ab_immobilized_assets 
    # денежные средства + краткосрочные фин. вложения:
    # почему-то тут оборотные активы = запасы + дебиторка + прочие, а денежные средства и фин. вложения где???
    # взял здесь поэтому просто денежные средства!!!
    #cash_and_short_investments = df.ab_mobile_current_assets	- (df.ab_inventory + df.ab_accounts_receivable + df.ab_other_current_assets)
    cash_and_short_investments = df.ab_cash_and_securities

    df['nf_1'] = (df.ab_mobile_current_assets - short_term_lialabilities) / assets
    df['nf_2'] = df.ab_losses / assets
    df['nf_3'] = df.ar_profit_before_tax / assets
    df['nf_4'] = df.ab_own_capital / (df.ab_long_term_liabilities + short_term_lialabilities)
    df['nf_5'] = df.ar_revenue / assets
    df['nf_6'] = cash_and_short_investments / assets
    df['nf_7'] = df.ar_revenue / cash_and_short_investments
    df['nf_8'] = (df.ab_long_term_liabilities + short_term_lialabilities) / assets
    df['nf_9'] = (df.ab_mobile_current_assets - short_term_lialabilities) / df.ar_revenue
    df['nf_10'] = cash_and_short_investments / short_term_lialabilities
    df['nf_11'] = (cash_and_short_investments + df.ab_accounts_receivable) / short_term_lialabilities
    df['nf_12'] = df.ab_mobile_current_assets / short_term_lialabilities
    df['nf_13'] = ((df.ab_own_capital + df.ab_long_term_liabilities) - df.ab_immobilized_assets) / df.ab_mobile_current_assets
  
    return df

In [ ]:
# Добавим новые фичи в датасет:

X_0 = get_new_features(X_0)
X_0.head()

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,nf_1,nf_2,nf_3,nf_4,nf_5,nf_6,nf_7,nf_8,nf_9,nf_10,nf_11,nf_12,nf_13
0,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,0.0,198,136,3,0,50000.0,2,-1.192265,0.165470,0.282320,0.041772,7.319890,0.035221,207.827451,2.192265,-0.162880,0.016066,0.157951,0.456149,0.091575
3,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,0.0,185,139,8,0,300000.0,1,0.139694,0.452061,0.523204,0.607612,7.549823,0.005889,1281.960000,0.860306,0.018503,0.006846,0.051479,1.162377,0.522733
4,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,0.0,220,138,27,0,10000.0,2,0.857095,0.079441,0.139553,9.725567,1.838883,0.054972,33.451220,0.142905,0.466096,0.384676,0.653636,6.997654,1.389832
9,125432000.0,122655000.0,122655000.0,0.0,0.0,2777000.0,-63000.0,2714000.0,579000.0,0.0,2135000.0,0.0,24680000.0,8263000.0,16417000.0,0.0,106000.0,2135000.0,11267000.0,11912000.0,0.0,13000.0,11899000.0,0.0,170,132,15,1,10000.0,2,0.517342,0.086507,0.109968,0.945853,5.082334,0.004295,1183.320755,0.482658,0.101792,0.008899,1.387089,2.071860,0.456524
10,59570000.0,59331000.0,59331000.0,0.0,0.0,239000.0,-97000.0,142000.0,38000.0,0.0,104000.0,0.0,38004000.0,23189000.0,14815000.0,0.0,449000.0,104000.0,10000.0,38443000.0,0.0,4986000.0,33457000.0,0.0,29,29,4,0,10000.0,2,-0.011551,0.002737,0.003736,0.000260,1.567467,0.011815,132.672606,1.011551,-0.007369,0.011680,0.397055,0.988580,0.000263


Видим, что в 1600 строчках появились пропуски и бесконечности. Это связано с тем, что при расчете показателей знаменатель (или +числитель) обнуляется, т.к. в финансовой отчетности много нулей (в некоторых строчках - все):

In [ ]:
X_0[X_0.replace([np.inf, -np.inf], np.nan).isna().any(axis=1)]

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,ogrn_age,adr_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,nf_1,nf_2,nf_3,nf_4,nf_5,nf_6,nf_7,nf_8,nf_9,nf_10,nf_11,nf_12,nf_13
14,23142000.0,22064000.0,22064000.0,0.0,0.0,1078000.0,-9000.0,1069000.0,214000.0,0.0,855000.0,0.0,11564000.0,8770000.0,2794000.0,0.0,606000.0,855000.0,865000.0,0.0,0.0,0.0,0.0,0.0,20,20,16,0,10000.0,1,1.000000,0.073936,0.092442,inf,2.001211,0.052404,38.188119,0.000000,0.499698,inf,inf,inf,0.074801
16,860000.0,274000.0,146000.0,50000.0,78000.0,586000.0,-586000.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0,0.0,0.0,150000.0,0.0,10000.0,150000.0,0.0,150000.0,0.0,0.0,20,20,4,0,10000.0,1,-15.000000,0.000000,0.000000,0.066667,86.000000,15.000000,5.733333,15.000000,-0.174419,1.000000,1.000000,0.000000,NaN
21,4654000.0,4619000.0,4619000.0,0.0,0.0,35000.0,-5000.0,30000.0,6000.0,0.0,24000.0,0.0,266000.0,0.0,266000.0,0.0,0.0,24000.0,34000.0,232000.0,0.0,232000.0,0.0,0.0,32,32,27,0,10000.0,1,0.127820,0.090226,0.112782,0.146552,17.496241,0.000000,inf,0.872180,0.007306,0.000000,1.146552,1.146552,0.127820
27,13048000.0,13003000.0,13003000.0,0.0,0.0,45000.0,-34000.0,11000.0,2000.0,0.0,9000.0,0.0,1576000.0,1576000.0,0.0,0.0,2436000.0,9000.0,11000.0,0.0,0.0,0.0,0.0,0.0,48,48,26,0,10000.0,5,1.000000,0.005711,0.006980,inf,8.279188,1.545685,5.356322,0.000000,0.120785,inf,inf,inf,0.006980
39,169000.0,113000.0,113000.0,0.0,0.0,56000.0,0.0,56000.0,0.0,0.0,56000.0,0.0,47000.0,47000.0,0.0,0.0,19000.0,56000.0,66000.0,0.0,0.0,0.0,0.0,0.0,17,17,3,0,10000.0,1,1.000000,1.191489,1.191489,inf,3.595745,0.404255,8.894737,0.000000,0.278107,inf,inf,inf,1.404255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32249,787000.0,1272000.0,1272000.0,0.0,0.0,-485000.0,0.0,-485000.0,0.0,0.0,-485000.0,0.0,0.0,0.0,0.0,0.0,0.0,-485000.0,0.0,0.0,0.0,0.0,0.0,0.0,98,98,17,0,50000.0,1,NaN,-inf,-inf,NaN,inf,NaN,inf,NaN,0.000000,NaN,NaN,NaN,NaN
32306,4615000.0,4609000.0,4609000.0,0.0,0.0,6000.0,0.0,6000.0,0.0,0.0,6000.0,0.0,3915000.0,3915000.0,0.0,0.0,0.0,6000.0,878000.0,0.0,0.0,0.0,0.0,0.0,94,94,7,0,10000.0,1,1.000000,0.001533,0.001533,inf,1.178799,0.000000,inf,0.000000,0.848321,NaN,NaN,inf,0.224266
32309,2591000.0,1964000.0,1964000.0,0.0,0.0,627000.0,0.0,627000.0,38000.0,0.0,589000.0,0.0,86000.0,0.0,86000.0,0.0,0.0,589000.0,4493000.0,402000.0,0.0,300000.0,102000.0,0.0,237,138,14,0,3672102.0,1,-3.674419,6.848837,7.290698,11.176617,30.127907,0.000000,inf,4.674419,-0.121961,0.000000,0.213930,0.213930,52.244186
32347,5966000.0,5617000.0,5617000.0,0.0,0.0,349000.0,0.0,349000.0,0.0,0.0,349000.0,0.0,296000.0,296000.0,0.0,0.0,0.0,349000.0,-59000.0,355000.0,0.0,172000.0,183000.0,0.0,41,41,31,0,10000.0,1,-0.199324,1.179054,1.179054,-0.166197,20.155405,0.000000,inf,1.199324,-0.009889,0.000000,0.000000,0.833803,-0.199324


Было решено такие строки отнести к 1 группе данных (где вообще отсутствуют данные фин. отчетности).

In [ ]:
X_0.replace([np.inf, -np.inf], np.nan, inplace=True)

nans_X_0_bool = X_0.isna().any(axis=1)

to_concat = X_0.loc[nans_X_0_bool, train_df.columns[24:-1]]
to_concat_y = y_0[nans_X_0_bool]

X_1 = pd.concat([X_1, to_concat])
y_1 = pd.concat([y_1, to_concat_y])

y_0 = y_0.drop(X_0[nans_X_0_bool].index)
X_0 = X_0.drop(X_0[nans_X_0_bool].index)

Отфильтруем обе группы данных от выбросов при помощи Isolation Forest

In [ ]:
isol_forest = IsolationForest(n_estimators=250, random_state=0, n_jobs=-1)

isol_forest.fit(X_0)
is_outliers_0 = isol_forest.predict(X_0)
y_0 = y_0.drop(X_0[is_outliers_0==-1].index)
X_0 = X_0.drop(X_0[is_outliers_0==-1].index)

isol_forest.fit(X_1)
is_outliers_1 = isol_forest.predict(X_1)
y_1 = y_1.drop(X_1[is_outliers_1==-1].index)
X_1 = X_1.drop(X_1[is_outliers_1==-1].index)

print(f'''В группе X_0 отфильтровано {(is_outliers_0==-1).sum()} выбросов
В группе X_1 отфильтровано {(is_outliers_1==-1).sum()} выбросов
''')

В группе X_0 отфильтровано 565 выбросов
В группе X_1 отфильтровано 1580 выбросов



## Отбор признаков

Попытаемся для группы X_0 отобрать часть признаков, так как их стало довольно много (43). В X_1 всего 6 признаков, поэтому отбирать лишние особого смысла нет.

In [ ]:
# Сократим количество признаков с 43 до 30 на основе метода ReliefF

n_new_features = 30

feature_filter = ReliefF(n_features_to_select=n_new_features, n_neighbors=100, n_jobs=-1)
X_0_new = feature_filter.fit_transform(X_0.values, y_0.values)

In [ ]:
new_cols = []
cols_to_drop = []

for i in range(n_new_features):
    for col in X_0.columns:
        if (X_0.reset_index().loc[:, col] - pd.DataFrame(X_0_new).iloc[:, i]).sum() == 0:
            new_cols.append(col)

for col in X_0.columns:
    if col not in new_cols:
        cols_to_drop.append(col)

In [ ]:
cols_to_drop

['ar_selling_expenses',
 'ar_management_expenses',
 'ar_balance_of_rvns_and_expns',
 'ar_other_profit_and_losses',
 'ab_other_current_assets',
 'ab_cash_and_securities',
 'ab_other_borrowings',
 'ul_capital_sum',
 'nf_4',
 'nf_9',
 'nf_10',
 'nf_11',
 'nf_13']

In [ ]:
cols_to_drop = [
    'ar_selling_expenses', 'ar_management_expenses', 'ar_balance_of_rvns_and_expns',
    'ar_other_profit_and_losses', 'ab_other_current_assets', 'ab_cash_and_securities', 
    'ab_other_borrowings', 'ul_capital_sum', 'nf_4', 'nf_9', 'nf_10', 'nf_11', 'nf_13'
]

In [ ]:
X_0_new = X_0.drop(cols_to_drop, axis=1)
X_0_new.head()

,ar_revenue,ar_total_expenses,ar_sale_cost,ar_sale_profit,ar_profit_before_tax,ar_taxes,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_losses,ab_own_capital,ab_borrowed_capital,ab_long_term_liabilities,ab_short_term_borrowing,ab_accounts_payable,bus_age,ogrn_age,adr_actual_age,ul_staff_range,ul_founders_cnt,nf_1,nf_2,nf_3,nf_5,nf_6,nf_7,nf_8,nf_12
0,52996000.0,48344000.0,48344000.0,4652000.0,2044000.0,846000.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,1198000.0,663000.0,15872000.0,0.0,11349000.0,4523000.0,198,136,3,0,2,-1.192265,0.165470,0.282320,7.319890,0.035221,207.827451,2.192265,0.456149
3,32049000.0,29568000.0,29568000.0,2481000.0,2221000.0,302000.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,1919000.0,2219000.0,3652000.0,0.0,3652000.0,0.0,185,139,8,0,1,0.139694,0.452061,0.523204,7.549823,0.005889,1281.960000,0.860306,1.162377
4,16458000.0,14935000.0,14935000.0,1523000.0,1249000.0,538000.0,711000.0,0.0,8950000.0,8606000.0,344000.0,711000.0,12439000.0,1279000.0,0.0,0.0,1279000.0,220,138,27,0,2,0.857095,0.079441,0.139553,1.838883,0.054972,33.451220,0.142905,6.997654
9,125432000.0,122655000.0,122655000.0,2777000.0,2714000.0,579000.0,2135000.0,0.0,24680000.0,8263000.0,16417000.0,2135000.0,11267000.0,11912000.0,0.0,13000.0,11899000.0,170,132,15,1,2,0.517342,0.086507,0.109968,5.082334,0.004295,1183.320755,0.482658,2.071860
10,59570000.0,59331000.0,59331000.0,239000.0,142000.0,38000.0,104000.0,0.0,38004000.0,23189000.0,14815000.0,104000.0,10000.0,38443000.0,0.0,4986000.0,33457000.0,29,29,4,0,2,-0.011551,0.002737,0.003736,1.567467,0.011815,132.672606,1.011551,0.988580


## Подбор гиперпараметров

In [ ]:
NFOLDS = 5
SEED = 0

param_grid_xgb = {
    'n_estimators': np.arange(150, 325, 25),
    'max_depth': list(np.arange(5, 20)) + [None],
    'skip_drop': np.linspace(0, 0.5, 10),
    'rate_drop': np.linspace(0, 0.5, 10),
    'learning_rate': np.linspace(0, 0.7, 15),
    'subsample': np.linspace(0.7, 1, 7),
    'reg_alpha': np.linspace(0, 1, 11),
    'reg_lambda': np.linspace(0, 1, 11),
}

cv = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

clf_xgb_0 = XGBClassifier(random_state=SEED, booster='dart', colsample_bytree=1)
clf_xgb_1 = XGBClassifier(random_state=SEED, booster='dart', colsample_bytree=1)

grid_cv_xgb_0 = RandomizedSearchCV(clf_xgb_0, param_grid_xgb, scoring='balanced_accuracy', cv=cv, n_jobs=-1, n_iter=100, random_state=SEED)
grid_cv_xgb_1 = RandomizedSearchCV(clf_xgb_1, param_grid_xgb, scoring='balanced_accuracy', cv=cv, n_jobs=-1, n_iter=100, random_state=SEED)

In [ ]:
%%time
np.random.seed(SEED)
grid_cv_xgb_0.fit(X_0_new, y_0)

best_params_0 = grid_cv_xgb_0.best_params_

print(f'''XGBoost, группа 0
Оптимальные гиперпараметры:\n{grid_cv_xgb_0.best_params_}\n
Balanced accuracy на кросс-валидации:\n{grid_cv_xgb_0.best_score_}''')

XGBoost, группа 0
Оптимальные гиперпараметры:
{'subsample': 0.75, 'skip_drop': 0.3333333333333333, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.6000000000000001, 'rate_drop': 0.3333333333333333, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.7}

Balanced accuracy на кросс-валидации:
0.5875884632617971
CPU times: user 1min 12s, sys: 618 ms, total: 1min 13s
Wall time: 4h 13min 26s


In [ ]:
%%time
np.random.seed(SEED)
grid_cv_xgb_1.fit(X_1, y_1)

best_params_1 = grid_cv_xgb_1.best_params_

print(f'''XGBoost, группа 1
Оптимальные гиперпараметры:\n{grid_cv_xgb_1.best_params_}\n
Balanced accuracy на кросс-валидации:\n{grid_cv_xgb_1.best_score_}''')

XGBoost, группа 1
Оптимальные гиперпараметры:
{'subsample': 0.95, 'skip_drop': 0.4444444444444444, 'reg_lambda': 0.5, 'reg_alpha': 0.30000000000000004, 'rate_drop': 0.3333333333333333, 'n_estimators': 275, 'max_depth': 15, 'learning_rate': 0.49999999999999994}

Balanced accuracy на кросс-валидации:
0.5324217677224543
CPU times: user 1min 9s, sys: 189 ms, total: 1min 10s
Wall time: 37min 50s


### Построение модели на всех данных

In [ ]:
best_params_0 = {
    'subsample': 0.9, 
    'skip_drop': 0.25, 
    'reg_lambda': 0.5, 
    'reg_alpha': 0.3, 
    'rate_drop': 0.15, 
    'n_estimators': 225, 
    'max_depth': 14, 
    'learning_rate': 0.25
}


best_params_1 = {
    'subsample': 0.95, 
    'skip_drop': 0.444, 
    'reg_lambda': 0.5, 
    'reg_alpha': 0.34, 
    'rate_drop': 0.333, 
    'n_estimators': 275, 
    'max_depth': 15, 
    'learning_rate': 0.5
}


In [ ]:
clf_xgb_0.set_params(**best_params_0)
clf_xgb_0.fit(X_0_new.values, y_0);

clf_xgb_0.set_params(**best_params_1)
clf_xgb_1.fit(X_1.values, y_1);

### Формирование файла с ответом

In [ ]:
answ_df = pd.DataFrame()
answ_df['id'] = test_df['record_id']

Для начала проведем на тестовом датасете те же преобразования, что делали в тренировочном

In [ ]:
#test_df = pd.read_csv('PD-data-test.csv', sep=';')

In [ ]:
test_df.drop(to_drop, axis=1, inplace=True)

test_df['ul_staff_range'] = test_df['ul_staff_range'].apply(simple_staff_coder)

X_0_test = test_df.dropna()
X_1_test = test_df.loc[test_df.iloc[:, :24].isna().any(axis=1), test_df.columns[24:]]

X_0_test = get_new_features(X_0_test)

X_0_test.replace([np.inf, -np.inf], np.nan, inplace=True)

nans_X_0_test_bool = X_0_test.isna().any(axis=1)

to_concat_test = X_0_test.loc[nans_X_0_test_bool, test_df.columns[24:]]

X_1_test = pd.concat([X_1_test, to_concat_test])
X_0_test = X_0_test.drop(X_0_test[nans_X_0_test_bool].index)

X_0_test = X_0_test.drop(cols_to_drop, axis=1)

Откалибруем классификатор при помощи изотонической регрессии

In [ ]:
clf_xgb_0_calib = CalibratedClassifierCV(base_estimator=clf_xgb_0, cv='prefit', method='isotonic')
clf_xgb_1_calib = CalibratedClassifierCV(base_estimator=clf_xgb_1, cv='prefit', method='isotonic')

clf_xgb_0_calib.fit(X_0_new.values, y_0);
clf_xgb_1_calib.fit(X_1.values, y_1);

Теперь получим прогнозы и запишем ответ

In [ ]:
cutoff_0 = y_0.mean()
cutoff_1 = y_1.mean() 

preds_0 = (clf_xgb_0_calib.predict_proba(X_0_test.values)[:, 1]>cutoff_0).astype(int)
preds_1 = (clf_xgb_1_calib.predict_proba(X_1_test.values)[:, 1]>cutoff_1).astype(int)

answ_df.loc[X_0_test.index, 'predict'] = preds_0
answ_df.loc[X_1_test.index, 'predict'] = preds_1

In [ ]:
answ_df.to_csv('PD-submit.csv',index=False, sep=';')